## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kodiak,US,57.7900,-152.4072,37.06,77,100,12.33,overcast clouds
1,1,Gravdal,NO,68.1214,13.5031,29.68,88,100,11.77,overcast clouds
2,2,Busselton,AU,-33.6500,115.3333,68.52,60,46,20.54,scattered clouds
3,3,Mataura,NZ,-46.1927,168.8643,69.58,49,0,1.95,clear sky
4,4,Boguchany,RU,58.3814,97.4531,14.95,86,100,13.27,overcast clouds
5,5,New Norfolk,AU,-42.7826,147.0587,74.70,40,0,2.57,clear sky
6,6,Atuona,PF,-9.8000,-139.0333,79.39,79,3,15.97,clear sky
7,7,Bredasdorp,ZA,-34.5322,20.0403,61.11,83,5,8.57,clear sky
8,8,Punta Arenas,CL,-53.1500,-70.9167,46.11,93,100,9.89,overcast clouds
9,9,Grayvoron,RU,50.4767,35.6773,33.62,74,8,18.32,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Atuona,PF,-9.8000,-139.0333,79.39,79,3,15.97,clear sky
10,10,Hilo,US,19.7297,-155.0900,78.53,79,100,14.97,overcast clouds
13,13,Laguna,US,38.4210,-121.4238,75.34,37,0,10.36,clear sky
19,19,Grand Gaube,MU,-20.0064,57.6608,79.38,82,8,3.00,clear sky
20,20,Souillac,MU,-20.5167,57.5167,77.38,94,75,0.00,broken clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No empty roads otherwise use dropna function

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Atuona,PF,79.39,clear sky,-9.8000,-139.0333,
10,Hilo,US,78.53,overcast clouds,19.7297,-155.0900,
13,Laguna,US,75.34,clear sky,38.4210,-121.4238,
19,Grand Gaube,MU,79.38,clear sky,-20.0064,57.6608,
20,Souillac,MU,77.38,broken clouds,-20.5167,57.5167,
21,Geraldton,AU,79.21,broken clouds,-28.7667,114.6000,
22,Rikitea,PF,79.63,clear sky,-23.1203,-134.9692,
26,Khandwa,IN,83.26,clear sky,21.8333,76.3333,
32,Butaritari,KI,81.46,scattered clouds,3.0707,172.7902,
35,Pijijiapan,MX,77.14,overcast clouds,15.7011,-93.2300,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('No hotel found... skipping!')
    
        

No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna().count()

City                   184
Country                184
Max Temp               184
Current Description    184
Lat                    184
Lng                    184
Hotel Name             184
dtype: int64

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [19]:
 # 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


In [20]:
#11b display the figure
fig

Figure(layout=FigureLayout(height='420px'))